# Benchmarking Our Algorithm against others

In [1]:
import sys
sys.path.append('../')
from panav.environment.env import WareHouse
from panav.hybrid import HybridGraph
from panav.util import flowtime,makespan
from panav.PBS.PBS import PBS
from panav.TrafficAwarePlanning import TAHP
from panav.sequential import sequential_planning
from panav.PIBT import PIBT_plan
from panav.SAMP.solvers import Tube_Planning



from time import time
import pickle as pkl

%load_ext autoreload
%autoreload 2

In [5]:

bloating_r = 0.5
vmax = 1.0

#### 100 agent configuration ####
n_agents = list(range(10,101,10))
bloating_r = 0.5
vmax = 1.0

n_col = 10
n_row = 10

limits = [(-50,50),(-50,50)] 

corner_padding_x = bloating_r * 10
corner_padding_y = bloating_r * 10

TIMEOUT = 100 # timeout

# Algorithms
algs = {  
        "PIBT": lambda hg: PIBT_plan(hg,vmax,bloating_r,TIMEOUT),
        "S2M2": lambda hg: PBS(hg.env,vmax=vmax,bloating_r=bloating_r,low_level_planner="Simple_MILP_Planning",TIMEOUT=TIMEOUT)[0],
        "PBS_Tube": lambda hg: PBS(hg.env,vmax=vmax,bloating_r=bloating_r,low_level_planner="Tube_Planning",TIMEOUT=TIMEOUT)[0],
        "TAHP": lambda hg: TAHP(hg,vmax=vmax,bloating_r=bloating_r,TIMEOUT=TIMEOUT),
        "Seq_Tube": lambda hg: sequential_planning(Tube_Planning,hg.env,vmax,bloating_r,TIMEOUT=TIMEOUT),      
        # "Seq_Temp_Obs": lambda hg: sequential_planning(Simple_MILP_Planning,hg.env,vmax,bloating_r,TIMEOUT=TIMEOUT)
        }

timeouted = {alg:False for alg in algs}

results = []

env_name = "Warehouse"


In [6]:
# Experiments
cur_N = 0
for i in range(len(n_agents)):
    
    N = n_agents[i]
   
    env = WareHouse(limits = limits, N_agent = N, 
                    shelf_region_x_limit=[-30,30],shelf_region_y_limit=[-30,30], 
                    obs_x_margin= 8 * bloating_r,
                    obs_y_margin= 8 * bloating_r,
                    n_col = n_col, n_row = n_row,
                    corner_padding_x=corner_padding_x,corner_padding_y=corner_padding_y)
    HG = HybridGraph(env, agent_radius = bloating_r,d = 2,vmax = vmax,tunnels=env.get_tunnels())



    if N<cur_N:
        continue
  
    # Save checkpoint data
    with open("./data/{}_misc.pkl".format(env_name),'wb') as fp:
        pkl.dump({"TIMEOUT":TIMEOUT,'cur_N':N},fp)
        
    for name,alg in algs.items():

        if timeouted[name]:
            continue
        
        print("Alg:", name, "N agent:",N)

        result = {}
        
        t0 = time()
        plan = alg(HG)
        t = time()-t0
        
        result['plan'] = plan
        result['time'] = t

        result['N_agent'] = N
        result['alg_name'] = name

        result["TIMEOUT"]=TIMEOUT
        result["HG"]=HG

        results.append(result)

        
        if t>TIMEOUT:
            timeouted[name] = True
            print("Time out for algorithm", name, "N agent = ", N)

    # Save checkpoint data
    with open("./data/{}.pkl".format(env_name),'wb') as fp:
        pkl.dump(results,fp)


Alg: PIBT N agent: 10
Alg: S2M2 N agent: 10
PBS Time Elapsed: 4.327664136886597
PBS Time Elapsed: 4.802728891372681
PBS Time Elapsed: 5.254217863082886
PBS Time Elapsed: 6.049261093139648
PBS Time Elapsed: 6.9933459758758545
PBS Time Elapsed: 7.4827470779418945


/Users/tianpengzhang/opt/miniconda3/envs/PA-NAV/lib/python3.11/site-packages/cvxpy/problems/problem.py:1385: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


PBS Time Elapsed: 371.4425060749054
Time out for algorithm S2M2 N agent =  10
Alg: PBS_Tube N agent: 10
PBS Time Elapsed: 5.085517168045044
PBS Time Elapsed: 5.7861409187316895
PBS Time Elapsed: 6.729047060012817
PBS Time Elapsed: 7.6722869873046875
PBS Time Elapsed: 8.36366319656372
PBS Time Elapsed: 9.621946096420288
PBS Time Elapsed: 10.745362997055054
PBS Time Elapsed: 12.01458215713501
PBS Time Elapsed: 12.983822107315063
PBS Time Elapsed: 14.000913143157959
PBS Time Elapsed: 14.79722809791565
PBS Time Elapsed: 15.543275117874146
PBS Time Elapsed: 16.326585054397583
PBS Time Elapsed: 17.015270233154297
PBS Time Elapsed: 17.992125988006592
PBS Time Elapsed: 18.710631132125854
PBS Time Elapsed: 19.398319244384766
PBS Time Elapsed: 20.70583701133728
PBS Time Elapsed: 21.410951852798462
PBS Time Elapsed: 22.124035120010376
PBS Time Elapsed: 23.417206048965454
PBS Time Elapsed: 24.150540113449097
PBS Time Elapsed: 25.495586156845093
PBS Time Elapsed: 26.525405883789062
PBS Time Elapsed